# Lab Assignment 1 - Part2 
## With this assignment you will get to know more about gradient descent optimization and writing your own functions with forward and backward (i.e., gradient) passes
## You need to complete all the tasks in this notebook in the lab and show you work to the TA. Edit only those portions in the cells where it asks you to do so!

In [1]:
import torch
from torch.autograd import Variable
from torch.autograd import Function
import torch.nn.functional as F
import numpy as np

## Huber loss function
https://en.wikipedia.org/wiki/Huber_loss

In [2]:
# A loss function measures distance between a predicted and a target tensor
# An implementation of Huber loss function is given below
# We will make use of this loss function in gradient descent optimization
def Huber_Loss(input,delta):
  m = (torch.abs(input)<=delta).detach().float()
  #print(m)  
  output = torch.sum(0.5*m*input**2 + delta*(1.0-m)*(torch.abs(input)-0.5*delta))
  return output

# Test Huber loss with a couple of different examples

In [3]:
a = torch.tensor([[0.3, 2.0, -3.1],[0.5, 9.2, 0.1]])
print(a.numpy())
ha = Huber_Loss(a,1.0)
print(ha.numpy())

b = torch.tensor([0.3, 2.0])
print(b.numpy())
hb = Huber_Loss(b,1.0)
print(hb.numpy())

[[ 0.3  2.  -3.1]
 [ 0.5  9.2  0.1]]
12.974999
[0.3 2. ]
1.545


# Gradient descent code
## Study the following generic gradient descent optimization code.
## Huber loss f measures the distance between a probability vector `z` and target 1-hot vector `target`.
## When `f.backward` is called, PyTorch first computes $\nabla_z f$ (gradient of `f` with respect to `z`), then by chain rule it computes $\nabla_{var} f = J^{z}_{var} \nabla_z f$, where $J^{z}_{var}$ is the Jacobian of `z` with respect to `var`.
## Next, `optimizer.step()` call adjusts the variable `var` in the opposite direction of $\nabla_{var} f.$

In [4]:
def gradient_descent(var,optimizer,softmax,loss,target,nIter,nPrint):
  for i in range(nIter):
    z = softmax(var)
    f = loss(z-target,1.0)
    optimizer.zero_grad()
    f.backward()
    optimizer.step()
    if i%nPrint==0:
      with np.printoptions(precision=3, suppress=True):
        print("Iteration:",i,"Variable:", z.detach().numpy(),"Loss: %0.6f" % f.item())


# Gradient descent with Huber Loss
## The following cell shows how `gradient_descent` function can be used.
## The cell first creates a target 1-hot vector `y`, where only the 3rd place is on.
## It also creates a variable `x` with random initialization and an optimizer.
## Learning rate and momentum has been set to 0.1 and 0.9, respectively.
## Then it calls `gradient_descent` function.

In [5]:
y = torch.zeros(10)
y[2] = 1.0
print("Target 1-hot vector:",y.numpy())
x = Variable(torch.randn(y.shape),requires_grad=True)

optimizer = torch.optim.SGD([x], lr=1e-1, momentum=0.9) # create an optimizer that will do gradient descent optimization

gradient_descent(x,optimizer,F.softmax,Huber_Loss,y,1000,100)


Target 1-hot vector: [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
Iteration: 0 Variable: [0.087 0.011 0.195 0.233 0.016 0.024 0.239 0.15  0.014 0.03 ] Loss: 0.396109
Iteration: 100 Variable: [0.008 0.001 0.952 0.01  0.002 0.003 0.01  0.01  0.002 0.003] Loss: 0.001329
Iteration: 200 Variable: [0.006 0.001 0.962 0.008 0.002 0.002 0.008 0.007 0.001 0.003] Loss: 0.000832
Iteration: 300 Variable: [0.005 0.001 0.968 0.006 0.001 0.002 0.006 0.006 0.001 0.002] Loss: 0.000606
Iteration: 400 Variable: [0.005 0.001 0.971 0.006 0.001 0.002 0.006 0.006 0.001 0.002] Loss: 0.000477


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  This is separate from the ipykernel package so we can avoid doing imports until


Iteration: 500 Variable: [0.004 0.001 0.974 0.005 0.001 0.002 0.005 0.005 0.001 0.002] Loss: 0.000393
Iteration: 600 Variable: [0.004 0.001 0.976 0.005 0.001 0.002 0.005 0.005 0.001 0.002] Loss: 0.000334
Iteration: 700 Variable: [0.004 0.001 0.978 0.004 0.001 0.001 0.004 0.004 0.001 0.002] Loss: 0.000290
Iteration: 800 Variable: [0.003 0.001 0.979 0.004 0.001 0.001 0.004 0.004 0.001 0.002] Loss: 0.000257
Iteration: 900 Variable: [0.003 0.001 0.98  0.004 0.001 0.001 0.004 0.004 0.001 0.002] Loss: 0.000230


# <font color='red'>30% Weight:</font> In this markdown cell, using [math mode](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html), write gradient of Huber loss function: $output = \sum_i 0.5 m_i (input)^{2}_{i} + \delta (1-m_i)(|input_i|-0.5 \delta)$ with respect to $input.$ Treat $m_i$ to be independent of $input_i,$ because we replaced *if* control statement with $m_i.$
## Your solution <font color='red'>20% (complete formula)</font>: $\frac{\partial (output)}{\partial (input)_i} = ...$

**Partial Differentiation**

Let $y^{\hat{}}$ = $output$

Let $y_{i}$ = $input_{i}$

$y^{\hat{}} = \sum_{i = 1}^{n} \frac{1}{2} m_{i} (y_{i})^2 + \delta (1-m_{i})(|y_{i}| - \frac{1}{2}\delta )$

$\frac{\partial y^{\hat{}} }{\partial y_{i}} = m_{i}y_{i} + \delta (1-m_{i})(\frac{y_{i}}{|y_{i}|})$

A much better way is to use the sgn function as whenever $y_{i} \neq 0$ we have 

${sgn}(x)={x \over |x|}={|x| \over x}$
Therefore
$\frac{\partial y^{\hat{}} }{\partial y_{i}} = m_{i}y_{i} + \delta (1-m_{i})({sgn}(y_{i}))$



# <font color='red'>30% Weight:</font> Define your own (correct!) rule of differentiation for Huber loss function
## Edit indicated line in the cell below. Use the following formula. Do not use for/while/any loop in your solution.
## For this function,  chain rule (Jacobian-vector product) takes the following form: $\frac{\partial (cost)}{\partial (input)_i} = \frac{\partial (output)}{\partial (input)_i} \frac{\partial (cost)}{\partial (output)}.$
# In the `backward` method below, $\frac{\partial (cost)}{\partial (output)}$ is denoted by `output_grad` and the $i^{th}$ component of `input_grad` is symbolized by $\frac{\partial (cost)}{\partial (input)_i}.$

In [6]:
# Inherit from torch.autograd.Function
class My_Huber_Loss(Function):

    # Note that both forward and backward are @staticmethods
    @staticmethod
    def forward(ctx, input, delta):
        m = (torch.abs(input)<=delta).float()
        ctx.save_for_backward(input,torch.tensor(m),torch.tensor(delta))
        output = torch.sum(0.5*m*input**2 + delta*(1.0-m)*(torch.abs(input)-0.5*delta))
        return output

    @staticmethod
    def backward(ctx, output_grad):
        # retrieve saved tensors and use them in derivative calculation
        input, m, delta = ctx.saved_tensors

        """
        Return Jacobian-vector product (chain rule)
        For Huber loss function the Jacobian happens to be a diagonal matrix
        Also, note that output_grad is a scalar, because forward function returns a scalar value
        """

        """
        As obtained in the above cell by differentiation and using the retrieved saved tensors
        Multiply this result by the output_grad to gain back the input_grad by the above formula(chain rule) 
        """
        input_grad = (m*input + delta*(1-m)*torch.sign(input))*output_grad 
        """
        must return two gradients becuase forward function takes in two arguments
        """
        return input_grad, None

#Gradient Descent on Your Own Huber Loss
## You should get almost identical results as before if your rule of differentation is correct!

In [10]:
y = torch.zeros(10)
y[2] = 1.0
print("Target:",y.numpy())
x = Variable(torch.randn(y.shape),requires_grad=True)

optimizer = torch.optim.SGD([x], lr=1e-1, momentum=0.9) # create an optimizer that will do gradient descent optimization

gradient_descent(x,optimizer,F.softmax,My_Huber_Loss.apply,y,1000,100)


Target: [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
Iteration: 0 Variable: [0.047 0.036 0.037 0.036 0.094 0.142 0.279 0.123 0.126 0.081] Loss: 0.538446
Iteration: 100 Variable: [0.005 0.004 0.941 0.004 0.007 0.008 0.008 0.008 0.008 0.007] Loss: 0.001919
Iteration: 200 Variable: [0.004 0.003 0.957 0.003 0.005 0.006 0.006 0.006 0.006 0.005] Loss: 0.001035
Iteration: 300 Variable: [0.003 0.003 0.964 0.003 0.004 0.005 0.005 0.005 0.005 0.004] Loss: 0.000717
Iteration: 400 Variable: [0.003 0.002 0.969 0.002 0.004 0.004 0.004 0.004 0.004 0.004] Loss: 0.000549
Iteration: 500 Variable: [0.002 0.002 0.972 0.002 0.004 0.004 0.004 0.004 0.004 0.003] Loss: 0.000444
Iteration: 600 Variable: [0.002 0.002 0.974 0.002 0.003 0.003 0.003 0.003 0.003 0.003] Loss: 0.000373
Iteration: 700 Variable: [0.002 0.002 0.976 0.002 0.003 0.003 0.003 0.003 0.003 0.003] Loss: 0.000321
Iteration: 800 Variable: [0.002 0.002 0.978 0.002 0.003 0.003 0.003 0.003 0.003 0.003] Loss: 0.000282


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


Iteration: 900 Variable: [0.002 0.002 0.979 0.002 0.003 0.003 0.003 0.003 0.003 0.003] Loss: 0.000251


# <font color='red'>40% Weight:</font> Your own softmax with forward and backward functions
## Edit indicated line in the cell below. Use the following formula. Do not use for/while/any loop in your solution.
## The Jacobian-vector product (chain rule) takes the following form using summation sign: $\frac{\partial (cost)}{\partial (input)_i} = \sum_j \frac{\partial (output)_j}{\partial (input)_i} \frac{\partial (cost)}{\partial (output)_j}$
# Once again note that, in the `backward` method below, $i^{th}$ component of `input_grad` and $j^{th}$ component of `output_grad` are denoted by $\frac{\partial (cost)}{\partial (input)_i}$ and $\frac{\partial (cost)}{\partial (output)_j}$, respectively.

In [8]:
# Inherit from Function
class My_softmax(Function):

    # Note that both forward and backward are @staticmethods
    @staticmethod
    def forward(ctx, input):
        output = F.softmax(input,dim=0)
        ctx.save_for_backward(output) # this is the only tensor you will need to save for backward function
        return output

    # This function has only a single output, so it gets only one gradient
    @staticmethod
    def backward(ctx, output_grad):
        output = ctx.saved_tensors[0]
        #print(output.dtype)
        # retrieve saved tensors and use them in derivative calculation
        # return Jacobian-vecor product

        matrix = np.outer(output,output)
        output_product = torch.tensor(matrix,dtype = torch.float32)
        """
        torch.diag = gives the diagonal of the tensor, offset = 0 returns the main diagonal 
        torch.ger = gives the outer product of two matrices
        torch.sum = returns the sum of the input elements in the tensor along the specified dimension
        dimnesion = 1 as returning the sum along the column
        """
        

        """
        Numpy outer product is much faster than torch.ger 
        When checking the results - numpy outer product however yields a bigger difference from the actual values
        reference = https://stackoverflow.com/questions/54357836/numpy-is-faster-than-pytorch-for-larger-cross-or-outer-products
        """
        # input_grad = torch.sum((torch.diag(output, 0) - output_product) *output_grad, dim = 1)

        """
        Now the reason why we are taking the diagonal matrix is because when i =\= j the partial derivative of the softmax function
        i =\= j : '0' - outer product of the product n * n matrix as the main diagonal contains all 0 entries
        i = j : diagnonal entries of the matrix - the outer product 
        This in turn gets multiplied by the output_grad by the chain rule
        Along each column the sum is taken with respect to each i thus dim = 1
        """
        input_grad = torch.sum((torch.diag(output, 0) - torch.ger(output, output)) *output_grad, dim = 1) 
        return input_grad

# Gradient Descent on your own Huber Loss and your own softmax

In [11]:
y = torch.zeros(10)
y[2] = 1.0
print(y)
x = Variable(torch.randn(y.shape),requires_grad=True)
print(x)

optimizer = torch.optim.SGD([x], lr=1e-1, momentum=0.9) # create an optimizer that will do gradient descent optimization

gradient_descent(x,optimizer,My_softmax.apply,My_Huber_Loss.apply,y,1000,100)


tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])
tensor([-0.4053,  1.7032, -0.0622, -0.7004,  1.0270, -0.7382, -1.3955,  1.5659,
        -0.1109,  1.5855], requires_grad=True)
Iteration: 0 Variable: [0.031 0.253 0.043 0.023 0.129 0.022 0.011 0.221 0.041 0.225] Loss: 0.549616
Iteration: 100 Variable: [0.004 0.009 0.947 0.003 0.009 0.003 0.002 0.009 0.005 0.009] Loss: 0.001609
Iteration: 200 Variable: [0.003 0.007 0.959 0.002 0.007 0.002 0.001 0.007 0.004 0.007] Loss: 0.000935
Iteration: 300 Variable: [0.003 0.006 0.966 0.002 0.006 0.002 0.001 0.006 0.003 0.006] Loss: 0.000664
Iteration: 400 Variable: [0.002 0.005 0.97  0.002 0.005 0.002 0.001 0.005 0.003 0.005] Loss: 0.000514
Iteration: 500 Variable: [0.002 0.004 0.973 0.002 0.005 0.002 0.001 0.005 0.003 0.004] Loss: 0.000420
Iteration: 600 Variable: [0.002 0.004 0.975 0.002 0.004 0.002 0.001 0.004 0.003 0.004] Loss: 0.000354


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


Iteration: 700 Variable: [0.002 0.004 0.977 0.001 0.004 0.001 0.001 0.004 0.002 0.004] Loss: 0.000307
Iteration: 800 Variable: [0.002 0.004 0.978 0.001 0.004 0.001 0.001 0.004 0.002 0.004] Loss: 0.000270
Iteration: 900 Variable: [0.002 0.003 0.979 0.001 0.003 0.001 0.001 0.003 0.002 0.003] Loss: 0.000241
